In [ ]:
import pandas as pd
%matplotlib inline

from entities import CoachingPracticeFinance, ExpenseLineItem, HoursLineItem

practice = CoachingPracticeFinance.load("../practice.json")

In [ ]:
period_start = pd.to_datetime("2022-11-01")
period_end = pd.to_datetime("2023-10-31")

report_dates = pd.bdate_range(period_start, period_end, freq="C", holidays=practice.statutory_holiday_list)

rows = []
for date in report_dates:
    for aa in practice.area_assignments:
        assignment_start = pd.to_datetime(aa.start_date)
        assignment_end = pd.to_datetime(aa.end_date)
        if assignment_start <= date <= assignment_end:
            for a in practice.support_areas:
                if aa.support_area_code == a.code:
                    for p in practice.contractors:
                        if p.code == aa.person_code:
                            for ta in practice.transaction_agreements:
                                ta_start = pd.to_datetime(ta.start_date)
                                ta_end = pd.to_datetime(ta.end_date)
                                ta_days = len(pd.bdate_range(ta_start, ta_end, freq="C", holidays=practice.statutory_holiday_list))
                                if ta.contractor_code == p.code and ta_start <= date <= ta_end:
                                    rows.append([
                                        date, a.code, aa.level, p.name, float(ta.rate.root) * ta.hours / ta_days
                                    ])
                    for p in practice.employees:
                        if p.code == aa.person_code:
                            rows.append([
                                date, a.code, aa.level, p.name, 15300*12 / len(report_dates)
                            ])
assignments = pd.DataFrame(rows, columns=["date", "area_code", "level", "person_name", "daily_rate"])
assignments['year'] = assignments.date.dt.year
assignments['month'] = assignments.date.dt.month
assignments.rename(columns={'daily_rate': 'budget_amount'}, inplace=True)
assignments

In [ ]:
#assignments[assignments['level'] == 'program']\
assignments\
    .groupby(['year', 'month', 'area_code', 'level'])\
    .agg({'budget_amount': 'sum', 'date': 'first'})\
    .reset_index()[['date', 'area_code', 'level', 'budget_amount']]\
    .set_index('date')

In [ ]:
rows = []
for c in practice.consultancies:
    for i in c.invoices:
        for l in i.line_items:
            tag = l.model_dump()['tag']
            if tag == "Hours":
                dates = pd.bdate_range(l.period_start, l.period_end, freq="C", holidays=practice.statutory_holiday_list)
                for date in dates:
                    person = None
                    ass = []
                    for p in practice.contractors:
                        if p.code == l.contractor_code:
                            for aa in practice.area_assignments:
                                if aa.person_code == p.code:
                                    person = p
                                    ass.append(aa)
                    if person is None:
                        raise Exception(f"No person found for contractor code {c.contractor_code}")
                    if len(ass) == 0:
                        raise Exception(f"No assignments found for contractor code {c.contractor_code} on {date}")
                    # Divide the hours evenly across the assignments
                    for aa in ass:
                        rows.append([
                            date, aa.support_area_code, aa.level, person.name, float(l.amount.root) / len(dates) / len(ass)
                        ])
            elif tag == "Expense":
                person = None
                ass = []
                for p in practice.contractors:
                    if p.code == l.contractor_code:
                        for aa in practice.area_assignments:
                            if aa.person_code == p.code:
                                person = p
                                ass.append(aa)
                if person is None:
                    raise Exception(f"No person found for contractor code {c.contractor_code}")
                if len(ass) == 0:
                    raise Exception(f"No assignments found for contractor code {c.contractor_code} on {date}")
                # Divide the expense evenly across the assignments
                for aa in ass:
                    # what if i.issue_date is not a working day? gets lost
                    issue_date = pd.to_datetime(i.issue_date)
                    next_business_day = pd.bdate_range(issue_date, issue_date+pd.Timedelta(days=5), freq="C", holidays=practice.statutory_holiday_list)[0]
                    rows.append([
                        next_business_day, aa.support_area_code, aa.level, person.name, float(l.amount.root) / len(ass)
                    ])
invoiced_amounts = pd.DataFrame(rows, columns=["date", "area_code", "level", "person_name", "invoiced_amount"])
invoiced_amounts

In [ ]:
assignments.merge(invoiced_amounts, on=["date", "area_code", "level", "person_name"], how="left")\
    .groupby(['year', 'month', 'area_code', 'level'])\
    .agg({'budget_amount': 'sum', 'invoiced_amount': 'sum', 'date': 'first'})\
    .reset_index()[['date', 'area_code', 'level', 'budget_amount', 'invoiced_amount']]\
    .set_index('date')

In [ ]:
assignments.merge(invoiced_amounts, on=["date", "area_code", "level", "person_name"], how="left")\
    .groupby(['year', 'month', 'area_code', 'level', 'person_name'])\
    .agg({'budget_amount': 'sum', 'invoiced_amount': 'sum', 'date': 'first'})\
    .reset_index()[['date', 'area_code', 'level', 'person_name', 'budget_amount', 'invoiced_amount']]\
    .set_index('date')

In [ ]:
rows = []
employee_daily_rate = 15300*12/len(report_dates)
for date in report_dates:
    if date < pd.Timestamp.today().normalize():
        for e in practice.employees:
            ass = []
            for aa in practice.area_assignments:
                start_date = pd.to_datetime(aa.start_date)
                end_date = pd.to_datetime(aa.end_date)
                if start_date <= date <= end_date and aa.person_code == e.code:
                    ass.append(aa)
            for aa in ass:
                rows.append([
                    date, aa.support_area_code, aa.level, e.name, employee_daily_rate/len(ass)
                ])
employee_amounts = pd.DataFrame(rows, columns=["date", "area_code", "level", "person_name", "fte_amount"])
employee_amounts

In [ ]:
assignments.merge(invoiced_amounts, on=["date", "area_code", "level", "person_name"], how="left")\
    .merge(employee_amounts, on=["date", "area_code", "level", "person_name"], how="left")\
    .groupby(['year', 'month', 'area_code', 'level', 'person_name'])\
    .agg({'budget_amount': 'sum', 'invoiced_amount': 'sum', 'fte_amount': 'sum', 'date': 'first'})\
    .reset_index()[['date', 'area_code', 'level', 'person_name', 'budget_amount', 'fte_amount', 'invoiced_amount']]\
    .set_index('date')

In [ ]:
assignments.merge(invoiced_amounts, on=["date", "area_code", "level", "person_name"], how="left")\
    .merge(employee_amounts, on=["date", "area_code", "level", "person_name"], how="left")\
    .groupby(['year', 'month', 'area_code', 'level'])\
    .agg({'budget_amount': 'sum', 'invoiced_amount': 'sum', 'fte_amount': 'sum', 'date': 'first'})\
    .reset_index()[['date', 'area_code', 'level', 'budget_amount', 'fte_amount', 'invoiced_amount']]\
    .set_index('date')